### 디렉토리 생성

In [ ]:
import datetime as dt
import os 
import shutil
x=dt.datetime.now()
day=str(x.year)+'_0'+str(x.month)+'_'+str(x.day)

if not os.path.exists(day):
    os.makedirs(day)
    shutil.move('./remains.csv', './'+day+'/remains.csv')

### WIKI ID 추출

In [ ]:
import mkwikidata
import json
import pandas as pd
import time

In [ ]:
none_ticker={'BALL':'Q4034815','BF-B':'Q392221','BRK-B':'Q217583','KDP':'Q3116111','WTW':'Q21189883'}

In [ ]:
sp480=pd.read_csv('./'+day+'/remains.csv')
ticker_symbol=dict()
count=0
for ticker in sp480:
    query='''
    SELECT ?company

    WHERE {
    
    ?company p:P414[pq:P249 '%s'] .
    
    SERVICE wikibase:label{
        bd:serviceParam wikibase:language "en"  .
        }
    }
    '''%ticker
    try:
        query_result=mkwikidata.run_query(query,params={})
        ticker_symbol[ticker]=query_result['results']['bindings'][0]['company']['value'][31:]
    except:
        if ticker not in none_ticker.keys():
            print(ticker)
        else:
            ticker_symbol[ticker]=none_ticker[ticker]
    time.sleep(2)

In [ ]:
with open('./'+day+'/wiki_ID.json','w') as f:
    json.dump(ticker_symbol,f,indent=2)

### file 생성

In [ ]:
import json
import csv
import pandas as pd
with open("./"+day+"/wiki_ID.json") as f:
    ticker_entityID=json.load(f) 
data=[]
for key in ticker_entityID.keys():
    sample={}
    sample['ticker']=key
    sample['wiki_code']=ticker_entityID[key]
    data.append(sample)
df=pd.DataFrame(data,columns=['ticker','wiki_code'])
df.to_csv('./'+day+'/company_480.csv',index=False)

### triple 추출

In [ ]:
import awena
import pandas as pd
import json
import csv

In [ ]:
reverse_list={'P127':'P1830','P1830':'P127','P155':'P156','P156':'P155',
'P355':'P749','P749':'P355'}

In [ ]:
with open("./"+day+"/wiki_ID.json") as f:
    ticker_entityID=json.load(f)

crawler		= awena.Crawler('en') # set language of labels and descriptions
data=[]
count=0
for ticker,wiki_code in ticker_entityID.items():
    
    info= crawler.load(wiki_code)
    for relation,node in info.items():
        # print(relation)
        if node=='None' or relation=='label' or relation=='id' or relation=='ticker':
            continue

        else:
            for i in node:
                sample=list()
                sample.extend([wiki_code,relation,i])
     
                if sample not in data:
                    data.append(sample)
            if relation in reverse_list.keys():
                for i in node:
                    sample=list()
                    sample.extend([i,reverse_list[relation],wiki_code])
                    if sample not in data:
                        data.append(sample)

data_dict=[]
for sample in data:
    triple={}
    triple['wiki_code']=sample[0]
    triple['relation']=sample[1]
    triple['node']=sample[2]
    data_dict.append(triple)
df=pd.DataFrame(data_dict,columns=['wiki_code','relation','node'])
df.to_csv('./'+day+'/triple.csv',index=False)



### Sector

In [ ]:
import pandas as pd
# path = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
path='https://en.wikipedia.org/w/index.php?title=List_of_S%26P_500_companies&oldid=1120818591'
companies = pd.read_html(path, flavor='bs4')[0].set_index('Symbol')[['GICS Sector']]
companies['temp']=companies.index
companies.loc[['BF.B'],['temp']]='BF-B'
companies.loc[['BRK.B'],['temp']]='BRK-B'
companies=companies.set_index('temp',drop=True)
companies=companies.rename_axis('Symbol')
with open('./'+day+'/wiki_ID.json', 'r') as fo:
    ticker = json.load(fo)

ticker=list(ticker.keys())
ticker.sort()
df=companies.loc[ticker]
df.rename(columns={"GICS Sector":"GICS"},inplace=True)
df.to_csv('./'+day+'/GICS.csv')
df.reset_index(inplace=True)
df.drop('Symbol',axis=1,inplace=True)
df.drop_duplicates(inplace=True)
df.to_csv('./'+day+'/GICS_type.csv',index=False)

In [ ]:
import pandas as pd
# path = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
path='https://en.wikipedia.org/w/index.php?title=List_of_S%26P_500_companies&oldid=1120818591'
companies = pd.read_html(path, flavor='bs4')[0].set_index('Symbol')[['GICS Sub-Industry']]
companies['temp']=companies.index
companies.loc[['BF.B'],['temp']]='BF-B'
companies.loc[['BRK.B'],['temp']]='BRK-B'
companies=companies.set_index('temp',drop=True)
companies=companies.rename_axis('Symbol')
# companies.drop(companies[companies['GICS Sub-Industry']=='Other Services'].index,inplace=True)
with open('./'+day+'/wiki_ID.json', 'r') as fo:
    ticker = json.load(fo)

ticker=list(ticker.keys())
ticker.sort()
df=companies.loc[ticker]

df_split1=df['GICS Sub-Industry'].str.split(' & ',expand=True)
df_split2=df_split1[0].str.split(', ',expand=True)
new=pd.concat([df_split1[1],df_split1[2],df_split2[0],df_split2[1]])
new=new.dropna().to_frame()
new.rename(columns={0:"GICS_SUB"},inplace=True)
new.drop(new[new['GICS_SUB']=='Other Services'].index,inplace=True)
new.to_csv('./'+day+'/GICS_SUB.csv')
new.reset_index(inplace=True)
new.drop('Symbol',axis=1,inplace=True)
new.drop_duplicates(inplace=True)
new.to_csv('./'+day+'/GICS_SUB_type.csv',index=False)

### Graph Generation

In [ ]:
from neo4j_driver import *

In [ ]:
query='''LOAD CSV WITH HEADERS FROM 'file:///%s/company_480.csv' as  line
create (:company{ticker:line.ticker,wiki_code:line.wiki_code,period:'%s'})
'''%(day,day)
run_query(query)

In [ ]:
query='''LOAD CSV WITH HEADERS FROM 'file:///%s/triple.csv' as line
merge (node:company{wiki_code:line.node,period:'%s'})
on create set node:not_company,node.wiki_code=line.node,node.period='%s'
'''%(day,day,day)
run_query(query)

In [ ]:
query='''match (n:not_company{period:'%s'})
remove n:company
'''%(day)
run_query(query)

In [ ]:
query='''LOAD CSV WITH HEADERS FROM 'file:///%s/triple.csv' as  line
match( node1:company{wiki_code:line.wiki_code,period:'%s'})
match( node2 {wiki_code:line.node,period:'%s'})
create (node1)-[:relation{relation:line.relation}]->(node2)
'''%(day,day,day)
run_query(query)

In [ ]:
query='''LOAD CSV WITH HEADERS FROM 'file:///%s/GICS.csv' as line
merge (node:GICS{field:line.GICS,period:'%s'})
on create set node:GICS,node.field=line.GICS,node.period='%s'
'''%(day,day,day)
run_query(query)

In [ ]:
query='''LOAD CSV WITH HEADERS FROM 'file:///%s/GICS_SUB.csv' as line
merge (node:GICS_SUB{field:line.GICS_SUB,period:'%s'})
on create set node:GICS_SUB,node.field=line.GICS_SUB,node.period='%s'
'''%(day,day,day)
run_query(query)

In [ ]:
query='''LOAD CSV WITH HEADERS FROM 'file:///%s/GICS.csv' as line
match (node1:company{ticker:line.Symbol,period:'%s'})
match (node2:GICS{field:line.GICS,period:'%s'})
create (node2)-[:GICS_R]->(node1)
'''%(day,day,day)
run_query(query)

In [ ]:
query='''LOAD CSV WITH HEADERS FROM 'file:///%s/GICS_SUB.csv' as line
match (node1:company{ticker:line.Symbol,period:'%s'})
match (node2:GICS_SUB{field:line.GICS_SUB,period:'%s'})
create (node2)-[:GICS_SUB_R]->(node1)
'''%(day,day,day)
run_query(query)

### Simple Graph 생성 - first

In [ ]:
from neo4j_driver import *
import pandas as pd
import numpy as np

In [ ]:
query='''LOAD CSV WITH HEADERS FROM 'file:///%s/company_480.csv' as line
match(node:company{ticker:line.ticker,period:'%s'})
with collect(node) as nodes
LOAD CSV WITH HEADERS FROM 'file:///data/first_order.csv' as line
with nodes,line
unwind nodes as node1
unwind nodes as node2
WITH line.r as edge_r,node1,node2,
CASE
    WHEN exists((node1)-[:relation{relation:line.r}]->(node2)) then 1
    ELSE 0
END AS overlap
return edge_r,collect(overlap)

'''%(day,day)
result=run_query_extraction(query)

In [ ]:
companies=pd.read_csv('./'+day+'/company_480.csv')
length=len(companies)
H=None
for line in result:
    if line['collect(overlap)'].count(1)<2:
        continue
    matrix=np.array(line['collect(overlap)']).reshape(length,length)
    if H is None:
        H=matrix[None,:]
    else:
        H=np.vstack((H,matrix[None,:]))
print(H.shape)
np.save('./'+day+'/first_simple.npy',H)

### Simple Graph 생성 - Second

In [ ]:
from neo4j_driver import *
import csv
import pandas as pd

In [ ]:
query='''LOAD CSV WITH HEADERS FROM 'file:///%s/company_480.csv' as line
match(node:company{ticker:line.ticker,period:'%s'})
with collect(node) as nodes
LOAD CSV WITH HEADERS FROM 'file:///data/second_not_arrange.csv' as line
with nodes,line
unwind nodes as node1
unwind nodes as node2
WITH line.r1 as edge_r1,line.r2 as edge_r2,node1,node2,
CASE
    WHEN exists((node1)-[:relation{relation:line.r1}]->()<-[:relation{relation:line.r2}]-(node2)) and node1.wiki_code <> node2.wiki_code then 1
    ELSE 0
END AS overlap
return edge_r1,edge_r2,collect(overlap)

'''%(day,day)
result=run_query_extraction(query)

In [ ]:
companies=pd.read_csv('./'+day+'/company_480.csv')
length=len(companies)
H=None
for line in result:
    if line['collect(overlap)'].count(1)<2:
        continue
    matrix=np.array(line['collect(overlap)']).reshape(length,length)
    if H is None:
        H=matrix[None,:]
    else:
        H=np.vstack((H,matrix[None,:]))
print(H.shape)
np.save('./'+day+'/second_simple.npy',H)

### Simple Graph - Sector

In [ ]:
from neo4j_driver import *
import numpy as np

In [ ]:
query=''' LOAD CSV WITH HEADERS FROM 'file:///%s/company_480.csv' as line
match(node:company{ticker:line.ticker,period:'%s'})
with collect(node) as nodes
LOAD CSV WITH HEADERS FROM 'file:///%s/GICS_type.csv' as line
with nodes,line
unwind nodes as node1
unwind nodes as node2
with line.GICS as gics,node1,node2,
case
    when node1.ticker<>node2.ticker and exists((:GICS{field:line.GICS,period:'%s'})-[:GICS_R]->(node1)) and exists(({field:line.GICS,period:'%s'})-[:GICS_R]->(node2)) then 1
    else 0
end as overlap
return gics,collect(overlap)

'''%(day,day,day,day,day)
result=run_query_extraction(query)

In [ ]:
companies=pd.read_csv('./'+day+'/company_480.csv')
length=len(companies)
H=None
for line in result:
    matrix=np.array(line['collect(overlap)']).reshape(length,length)
    if H is None:
        H=matrix[None,:]
    else:
        H=np.vstack((H,matrix[None,:]))
print(H.shape)
np.save('./'+day+'/sector_simple.npy',H)

### Hyper Graph - first

In [ ]:
from neo4j_driver import *
import csv
import pandas as pd

In [ ]:
query=''' LOAD CSV WITH HEADERS FROM 'file:///data/first_order.csv' as line
match (node1:company{period:'%s'})-[r:relation{relation:line.r}]->(node2:company{period:'%s'})
return node1.wiki_code,r.relation,node2.wiki_code
'''%(day,day)
result=run_query_extraction(query)

In [ ]:
data=list()
for line in result:
    sample=dict()
    sample['object']=line['node1.wiki_code']
    sample['r']=line['r.relation']
    data.append(sample)
df=pd.DataFrame(data,columns=['object','r'])
df.drop_duplicates(inplace=True)
df.to_csv('./'+day+'/first_hyperedge.csv',index=False)

In [ ]:
query=''' LOAD CSV WITH HEADERS FROM 'file:///%s/company_480.csv' as line
match(n:company{ticker:line.ticker,period:'%s'})
with collect(n) as nodes
LOAD CSV WITH HEADERS FROM 'file:///%s/first_hyperedge.csv' as line
with nodes,line
unwind nodes as node
optional match path=(:company{wiki_code:line.object,period:'%s'})-[:relation{relation:line.r}]->(node)
with line.object as edge_n,line.r as edge_r,path,
CASE
    WHEN path is null then 0
    ELSE 1
END AS overlap 
return edge_n,edge_r,collect(overlap)
'''%(day,day,day,day)
result=run_query_extraction(query)

In [ ]:
companies=pd.read_csv('./'+day+'/company_480.csv')
data=dict()
columns=list()
count_list=list()
count=0
for line in result:
    if line['collect(overlap)'].count(1)<2:
        count+=1
        continue
    else:
        count_list.append(line['collect(overlap)'].count(1))
    column=str(line['edge_n'])+':'+str(line['edge_r'])
    data[column]=line['collect(overlap)']
    columns.append(column)
df=pd.DataFrame(data,columns=columns,index=companies['ticker'])
df.to_csv('./'+day+'/first_hyper.csv')
print(count)

### Hyper Graph - Second

In [ ]:
from neo4j_driver import *
import csv
import pandas  as pd

In [ ]:
query=''' LOAD CSV WITH HEADERS FROM 'file:///data/second_order_arrange.csv' as line
match (node1:company{period:'%s'})-[r1:relation{relation:line.r1}]->(bridge{period:'%s'})<-[r2:relation{relation:line.r2}]-(node2:company{period:'%s'})
return r1.relation,r2.relation,bridge.wiki_code,node1.wiki_code,node2.wiki_code
order by r1.relation,r2.relation,bridge.wiki_code
'''%(day,day,day)
result=run_query_extraction(query)

In [ ]:
data=list()
for line in result:
    sample=dict()
    sample['r1']=line['r1.relation']
    sample['bridge']=line['bridge.wiki_code']
    sample['r2']=line['r2.relation']
    data.append(sample)
df=pd.DataFrame(data,columns=['r1','bridge','r2'])
df.drop_duplicates(inplace=True)
df.to_csv('./'+day+'/second_hyperedge.csv',index=False)

In [ ]:
day='2023_01_15'

In [ ]:
query=''' LOAD CSV WITH HEADERS FROM 'file:///%s/company_480.csv' as line
match(n:company{ticker:line.ticker,period:'%s'})
with collect(n) as nodes
LOAD CSV WITH HEADERS FROM 'file:///%s/second_hyperedge.csv' as line
with nodes,line
unwind nodes as node1 
with line.r1 as edge_r1,line.r2 as edge_r2,line.bridge as edge_n,node1,
CASE
    WHEN exists((node1)-[:relation{relation:line.r1}]->({wiki_code:line.bridge})<-[:relation{relation:line.r2}]-(:company)) then 1
    WHEN exists((:company)-[:relation{relation:line.r1}]->({wiki_code:line.bridge})<-[:relation{relation:line.r2}]-(node1)) then 1
    ELSE 0
END AS overlap 
return edge_r1,edge_r2,edge_n,collect(overlap)
'''%(day,day,day)
result=run_query_extraction(query)

In [ ]:
companies=pd.read_csv('./'+day+'/company_480.csv')
data=dict()
columns=list()
count_list=list()
count=0
for line in result:
    if line['collect(overlap)'].count(1)<2:
        count+=1
        continue
    else:
        count_list.append(line['collect(overlap)'].count(1))
    column=str(line['edge_r1'])+':'+str(line['edge_n'])+':'+str(line['edge_r2'])
    data[column]=line['collect(overlap)']
    columns.append(column)
df=pd.DataFrame(data,columns=columns,index=companies['ticker'])
df.to_csv('./'+day+'/second_hyper.csv')
print(count)

### Hyper Graph - Sector

In [ ]:
from neo4j_driver import *
import csv
import pandas  as pd

In [ ]:
query=''' LOAD CSV WITH HEADERS FROM 'file:///%s/company_480.csv' as line
match(node:company{ticker:line.ticker,period:'%s'})
with collect(node) as nodes
LOAD CSV WITH HEADERS FROM 'file:///%s/GICS_type.csv' as line
unwind nodes as node
with line.GICS as gics,node,
case
    when exists((:GICS{field:line.GICS,period:'%s'})-[:GICS_R]->(node)) then 1
    else 0
end as overlap
return gics,collect(overlap)
'''%(day,day,day,day)
result=run_query_extraction(query)

In [ ]:
companies=pd.read_csv('./'+day+'/company_480.csv')
data=dict()
columns=list()
for line in result:
    if line['collect(overlap)'].count(1)<2:
        continue
    data[line['gics']]=line['collect(overlap)']
    columns.append(line['gics'])
GICS=pd.DataFrame(data,columns=columns,index=companies['ticker'])

In [ ]:
query=''' LOAD CSV WITH HEADERS FROM 'file:///%s/company_480.csv' as line
match(node:company{ticker:line.ticker,period:'%s'})
with collect(node) as nodes
LOAD CSV WITH HEADERS FROM 'file:///%s/GICS_SUB_type.csv' as line
unwind nodes as node
with line.GICS_SUB as gics_sub,node,
case
    when exists((:GICS_SUB{field:line.GICS_SUB,period:'%s'})-[:GICS_SUB_R]->(node)) then 1
    else 0
end as overlap
return gics_sub,collect(overlap)
'''%(day,day,day,day)
result=run_query_extraction(query)

In [ ]:
companies=pd.read_csv('./'+day+'/company_480.csv')
data=dict()
columns=list()
for line in result:
    if line['collect(overlap)'].count(1)<2:
        continue
    data[line['gics_sub']]=line['collect(overlap)']
    columns.append(line['gics_sub'])
GICS_SUB=pd.DataFrame(data,columns=columns,index=companies['ticker'])

In [ ]:
SECTOR=pd.concat([GICS,GICS_SUB],axis=1,join='inner')
SECTOR.to_csv('./'+day+'/sector_hyper.csv')

### 중복제거

In [ ]:
def unique_graph(a):
    tmp = a.reshape(a.shape[0], -1)
    b = np.ascontiguousarray(tmp).view(np.dtype((np.void, tmp.dtype.itemsize * tmp.shape[1])))
    _, idx = np.unique(b, return_index=True)
    return a[idx].reshape(-1, *a.shape[1:])


def load_Graph_data(day):
    G,H=None,None
    try:
        g1=np.load('./'+day+'/sector_simple.npy')
        g2=np.load('./'+day+'/first_simple.npy')
        g3=np.load('./'+day+'/second_simple.npy')
        g123=np.concatenate((g1,g2,g3),0)
        G=unique_graph(g123)
        print('Simple graph shape: ',G.shape)
    except Exception:
        print('No simple graph')

    try:
        h1=pd.read_csv('./'+day+'/sector_hyper.csv',index_col='ticker')
        h2=pd.read_csv('./'+day+'/first_hyper.csv',index_col='ticker')
        h3=pd.read_csv('./'+day+'/second_hyper.csv',index_col='ticker')
        h123=h1.join(h2,how='left').join(h3,how='left')
        H=h123.loc[:,~h123.T.duplicated()].dropna(axis=1).values
        print('Hypergraph shape: ',H.shape)
    except Exception:
        print('No hypergraph')
    
    return G,H

In [ ]:
load_Graph_data(day)